## Part 4: Parse 2015 movies for prediction. 

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import requests
import json
import bs4
from bs4 import BeautifulSoup
import time

In [2]:
newmovies = pd.read_csv('new_movies.csv')
#tts = (list(set(movies.imdbID)))

In [3]:
tts = newmovies.imdb_id
tts2 = []
for t in tts:
    if str(t)!='nan':
        tts2.append(str(t))
tts = tts2

In [66]:
%%time
directorlist = []
for t in tts:
    url = "http://www.imdb.com/title/{}/".format(t)
    t1970 = requests.get(url)
    soup = BeautifulSoup(t1970.text, "html.parser")
    tb = soup.find("table").findAll("div", attrs = {'class','txt-block'})
    if tb[0].find('a')['href'] == None:
        directorlist.append(None)
    else:
        directorlist.append(tb[0].find('a')['href'])

CPU times: user 51.7 s, sys: 1.17 s, total: 52.8 s
Wall time: 3min 1s


In [67]:
#directorlist2 = dict(zip(tts, directorlist)) 
fd=open("directorlist2015.json","w")
json.dump(directorlist, fd)
fd.close()

In [73]:
with open("directorlist2015.json") as json_file:
    directorlist = json.load(json_file)

In [ ]:
%%time
director_dict = {"Name":[], "DOB":[], "Birthplace":[], "wins":[], "credits":[]}

for u in tqdm(directorlist, leave=True):
    #print u
    url = 'http://www.imdb.com{}'.format(u)
    t1990=requests.get(url)
    soup=BeautifulSoup(t1990.text, "html.parser")
    try:
        divs=soup.findAll('div', attrs = {'id':'name-born-info'})[0]
        Name = soup.findAll('table', attrs = {'id':'name-overview-widget-layout'})[0].find('span',attrs={'class':'itemprop'}).get_text()
        try: 
            DOB=divs.find('time').get('datetime')
        except:
            DOB=None 
        try:
            Birthplace=divs.findAll('a')[-1].get_text()
        except:
            Birthplace=None
        try:
            wins=soup.findAll('div', attrs = {'class':'article highlighted'})[0].find_all('span')[0].get_text().strip()
        except:
            wins=0
        try:
            credits=soup.findAll('div', attrs = {'id':'filmo-head-director'})[0].get_text().strip()[22:24]
        except:
            credits=0
        if Name not in director_dict["Name"]:
            director_dict["Name"].append(Name)
            director_dict['DOB'].append(DOB)
            director_dict['Birthplace'].append(Birthplace)
            director_dict['wins'].append(wins)
            director_dict['credits'].append(credits)
    except:
        pass
 

In [85]:
import datetime 
directordf = pd.DataFrame(director_dict) ### fix 'XXXX-0-0' date issue
for i in range(len(directordf.DOB)):
    if directordf.DOB[i] == None:
        directordf.DOB[i] = '2015-11-23'
        print directordf.DOB[i]
    if directordf.DOB[i].split('-')[1] == '0' or directordf.DOB[i].split('-')[2] == '0':
        directordf.DOB[i] = directordf.DOB[i].split('-')[0]+'-1'+'-1'
        print directordf.DOB[i]
        
today = datetime.datetime(2015, 11, 23)
directordf['age'] = (today - directordf.DOB.apply(pd.datetools.parse)).values/np.timedelta64(1, 'D')/365.25


1961-1-1
1970-1-1
2015-11-23
1972-1-1
1959-1-1
1952-1-1
2015-11-23
1967-1-1
1968-1-1
1957-1-1
1968-1-1
1974-1-1
2015-11-23
2015-11-23
1972-1-1
1965-1-1
1977-1-1
1973-1-1
1972-1-1
1979-1-1
1972-1-1
1965-1-1
1966-1-1
1963-1-1
1968-1-1
1973-1-1
1971-1-1
2015-11-23
1970-1-1
1964-1-1
2015-11-23
1976-1-1
1972-1-1


In [86]:
directordf.to_csv('directordf2015.csv', encoding='utf-8')

In [4]:
%%time
actorurl = {}
### return a dict, with key the movie id and values the urls for actors 
for t in tqdm(tts, leave=True):
    try:
        actorurl[t] = []
        l = []
        url = "http://www.imdb.com/title/{}/".format(t)
        t1970=requests.get(url)
        soup = BeautifulSoup(t1970.text, "html.parser")
        divs = soup.find('table').findAll('div', attrs = {'class':'txt-block', 'itemprop':'actors'})[0]
        for div in divs.find_all('a'):
            l.append(div.get('href'))
        actorurl[t] = l[:-1]
    except:
        pass

fd=open("actorurl2015.json","w")
json.dump(actorurl, fd)
fd.close()

100%|██████████| 305/305 [02:53<00:00,  2.02it/s]

CPU times: user 46.9 s, sys: 870 ms, total: 47.7 s
Wall time: 2min 53s


In [5]:
with open("actorurl2015.json") as json_file:
    actorurl = json.load(json_file)

In [6]:
%%time
actor_dict = {"Name":[], "DOB":[], "Birthplace":[], "wins":[], "credits":[]}

for k in tqdm(actorurl.keys(), leave=True):
    url_list = actorurl[k]
    for u in url_list:
        #print u
        try:
            url = 'http://www.imdb.com{}'.format(u)
            t1990=requests.get(url)
            soup=BeautifulSoup(t1990.text, "html.parser")
            divs=soup.findAll('table', attrs = {'id':'name-overview-widget-layout'})[0].findAll('div', attrs = {'id':'name-born-info'})[0]
            try:
                Name=soup.findAll('table', attrs = {'id':'name-overview-widget-layout'})[0].find('span',attrs={'class':'itemprop'}).get_text()
            except:
                Name=None
            try:
                DOB=divs.find('time').get('datetime')
            except:
                DOB=None
            try:
                Birthplace=divs.findAll('a')[-1].get_text()
            except:
                Birthdplace=None
            try:
                wins=soup.findAll('div', attrs = {'class':'article highlighted'})[0].find_all('span')[0].get_text().strip()
            except:
                wins=0
                
            try:
                c = soup.findAll('div', attrs = {'id':'filmo-head-actor'})[0]
                c = str(c.get_text()[20:].strip())
                credit = float(''.join(x for x in c if x.isdigit()))
            except:
                c = soup.findAll('div', attrs = {'id':'filmo-head-actress'})[0]
                c = str(c.get_text()[20:].strip())
                credit = float(''.join(x for x in c if x.isdigit()))
                
            if Name not in actor_dict["Name"]:
                actor_dict['Name'].append(Name)
                actor_dict['DOB'].append(DOB)
                actor_dict['Birthplace'].append(Birthplace)
                actor_dict['wins'].append(wins)
                actor_dict['credits'].append(credit)
        except:
            pass

import datetime
today = datetime.datetime(2015, 11, 23)
actordf=pd.DataFrame(actor_dict)

for i in range(len(actordf.DOB)):
    if actordf.DOB[i] == None:
        actordf.DOB[i] = '2015-11-23'
        print actordf.DOB[i]
    if actordf.DOB[i].split('-')[1] == '0' or actordf.DOB[i].split('-')[2] == '0':
        actordf.DOB[i] = actordf.DOB[i].split('-')[0]+'-1'+'-1'
        print actordf.DOB[i]
    
### calculate the age
actordf['age'] = (today - actordf.DOB.apply(pd.datetools.parse)).values/np.timedelta64(1, 'D')/365.25
actordf.to_csv('actordf2015.csv', encoding='utf-8')

100%|██████████| 304/304 [10:51<00:00,  0.53it/s]
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2015-11-23
1993-1-1
2015-11-23
1968-1-1
2015-11-23
1985-1-1
2015-11-23
1986-1-1
2015-11-23
2015-11-23
1947-1-1
2015-11-23
2015-11-23
1963-1-1
1963-1-1
1990-1-1
2015-11-23
2015-11-23
2015-11-23
2015-11-23
CPU times: user 3min 5s, sys: 3.44 s, total: 3min 8s
Wall time: 10min 52s


/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
